# BEMM459J - Database Technologies for Business Analytics

In [1]:
# Import library
import sqlite3
import pandas as pd

# Connect to the Database
conn=sqlite3.connect('invoicing.db')
cur=conn.cursor()
print(type(cur))
print (sqlite3.sqlite_version)

<class 'sqlite3.Cursor'>
3.33.0


## Show Tables

In [2]:
# List of all tables

def tables_in_sqlite_db(conn):
    cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [
        v[0] for v in cursor.fetchall()
        if v[0] != "sqlite_sequence"
    ]

    cursor.close()
    return tables

tables = tables_in_sqlite_db(conn)
#print tables
print(tables)

['Project', 'CompanyType', 'Company', 'Contact', 'ProductsServices', 'Invoice_ProductsServices', 'Invoice']


## CRUD operations (Create, Read, Update, Delete)


### Create

In [3]:
# Insert one record in table ECC_DEPARTMENT based on user input


inputcompanyRFC=input('Enter company identifier (RFC):')
inputprojectID=int(input('Enter Project number:'))
inputinvoiceDescription=input('Enter invoive description:')
inputinvoiceDate=input('Enter invoice date:')
inputinvoiceTotal=int(input('Enter invoice total:'))
inputinvoiceBalance=int(input('Enter paid amount:'))         


qry="insert into Invoice (companyRFC,projectID,invoiceDescription,invoiceDate,invoiceTotal,invoiceBalance) values (?,?,?,?,?,?);"

try:
        cur.execute(qry, (inputcompanyRFC,inputprojectID,inputinvoiceDescription,inputinvoiceDate,inputinvoiceTotal,inputinvoiceBalance))
        print ('New invoice successfully added')
        conn.commit()
except:
        print ('Error in adding invoice')
        conn.rollback()

Enter company identifier (RFC):
Enter Project number:


ValueError: invalid literal for int() with base 10: ''

In [ ]:
# Query and display tha last invoice created

# Query to show one invoice orderd by InvoiceID in desc order
qry="select * from Invoice order by invoiceID desc limit 1;"

# Execute the query on SQLite
cur.execute(qry)

# Fetch and display 
row=cur.fetchall()

print (row)

### Read

Show customer´s invoices and the amount they have to pay.  

In [ ]:
# Query and display records from two different tables for customers (not suppliers), finally add a new column with
#the difference of the invoice total and the amount that has been paid

# Prepare the query String
qry2=pd.read_sql_query("SELECT Company.companyName,Company.companyTypeID,Invoice.invoiceDescription,Invoice.invoiceTotal,Invoice.invoiceBalance, Invoice.invoiceTotal-Invoice.invoiceBalance AS TotalDueAmount FROM Company LEFT JOIN Invoice ON Invoice.companyRFC=Company.companyRFC WHERE Company.CompanyTypeID='Customer'",conn)

"""
SELECT Company.companyName,Company.companyTypeID,Invoice.invoiceDescription,
Invoice.invoiceTotal,Invoice.invoiceBalance, Invoice.invoiceTotal-Invoice.invoiceBalance AS TotalDueAmount
FROM Company 
LEFT JOIN Invoice ON Invoice.companyRFC=Company.companyRFC 
WHERE Company.CompanyTypeID="Customer"
"""
# Show the query ina Panda's Dataframe
qry2                       

In [4]:
# Query and display records from two different tables for customers (not suppliers),  add a new column with
#the difference of the invoice total and the amount that has been paid and SUM the Due Amount per Company

# Prepare the query String
qry3=pd.read_sql_query("SELECT Company.companyName,Company.companyTypeID,SUM(Invoice.invoiceTotal) AS 'Total Invoices',SUM(Invoice.invoiceBalance) AS 'Total Paid', SUM(Invoice.invoiceTotal-Invoice.invoiceBalance) AS 'Total Due Amount' FROM Company LEFT JOIN Invoice ON Invoice.companyRFC=Company.companyRFC WHERE Company.CompanyTypeID='Customer' GROUP BY companyName;",conn)

"""
SELECT Company.companyName,Company.companyTypeID,
SUM(Invoice.invoiceTotal) AS 'Total Invoices',SUM(Invoice.invoiceBalance) AS 'Total Paid', 
SUM(Invoice.invoiceTotal-Invoice.invoiceBalance) AS 'Total Due Amount'
FROM Company 
LEFT JOIN Invoice ON Invoice.companyRFC=Company.companyRFC 
WHERE Company.CompanyTypeID="Customer"
GROUP BY companyName
"""
# Show the query ina Panda's Dataframe
qry3

,companyName,CompanyTypeID,Total Invoices,Total Paid,Total Due Amount
0,"Formas e Imágenes, S.A. de C.V.",Customer,50000,25000,25000
1,"PLUS DIGITAL, SA DE CV",Customer,80032,80032,0
2,SERVICIO DE CONSULTORIA DE VALOR AGREGADO SC,Customer,614706,330000,284706
3,Víctor Adrián del Ángel Espinosa,Customer,75000,19000,56000


### Update

In [5]:
# We will Update the last invoice created
qry4=pd.read_sql_query("select * from Invoice order by invoiceID desc limit 1;",conn)
qry4

,invoiceID,companyRFC,projectID,invoiceDescription,invoiceDate,invoiceTotal,invoiceBalance,invoiceMethod,invoicetaxRate
0,16,S&S021003TBA,2,Sand,01/04/2021,10000,100,Wire Transfer,0.16


In [9]:
inputinvoiceID=int(input('Which invoice you want to pay?:'))
inputinvoiceAmount=int(input('Amount:'))


qry5='update Invoice set invoiceBalance=? where invoiceID=?'


try:
        cur.execute(qry5, (inputinvoiceAmount+100,inputinvoiceID))
        print ('Payment successfully registered')
        conn.commit()
except:
        print ('Error please check your info .. rollback')
        conn.rollback()

Which invoice you want to pay?:16
Amount:1000
Payment successfully registered


In [10]:
# Invoice Updated
qry6=pd.read_sql_query("select * from Invoice order by invoiceID desc limit 1;",conn)
qry6

,invoiceID,companyRFC,projectID,invoiceDescription,invoiceDate,invoiceTotal,invoiceBalance,invoiceMethod,invoicetaxRate
0,16,S&S021003TBA,2,Sand,01/04/2021,10000,1100,Wire Transfer,0.16


In [19]:
qr6="select Invoice.invoiceBalance from Invoice where invoiceID=?;"
current_balance=cur.execute(qr6,inputinvoiceID).fetchone()
current_balance=row[0]
q

1100

In [ ]:
inputinvoiceID=int(input('Which invoice you want to pay?:'))
inputinvoiceAmount=int(input('Amount:'))
inputinvoiceBalance

qry5='update Invoice set invoiceBalance=? where invoiceID=?'


try:
        cur.execute(qry5, (inputinvoiceAmount+inputinvoiceBalance,inputinvoiceID))
        print ('Payment successfully registered')
        conn.commit()
except:
        print ('Error please check your info .. rollback')
        conn.rollback()